In [24]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import json
import sqlite3

# Verbindung zur Datenbank herstellen
conn = sqlite3.connect('autoscout24.db')

# Tabelle erstellen, wenn sie nicht existiert
conn.execute('CREATE TABLE IF NOT EXISTS autos (id INTEGER PRIMARY KEY, model TEXT, price INTEGER, currency TEXT, year TEXT, address TEXT, mileage TEXT, fuel_type TEXT, engine_power INTEGER)')

# Set up the driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Define the base URL
base_url = 'https://www.autoscout24.ch/de/autos/audi--a5?make=5&uprop=2&model=27&vehtyp=10&page={}'

# Initialize an empty list to store data
data = []
page = 1
while True:
    # Build the URL for the current page
    url = base_url.format(page)

    # Navigate to the URL
    driver.get(url)

    # Check if "Kein Treffer" is found
    if "Keine Treffer" in driver.page_source:
        break

    # Increment the page number
    page += 1


    # Wait for the cookie button to be clickable and click it
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Akzeptieren")]'))
        )
        cookie_button.click()
    except:
        pass

    # Find the JSON data and extract the data
    json_elements = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//main//script[@type="application/ld+json"]')))
    for json_element in json_elements:
        json_data = json.loads(json_element.get_attribute('text'))
        for item in json_data['itemListElement']:
            if 'offers' in item['item']:
                price = item['item']['offers']['price']
                year  = item['item']['dateVehicleFirstRegistered']
                address = item['item']['offers']['seller']['address']
                currency = item['item']['offers']['priceCurrency']
                mileage = item['item']['mileageFromOdometer']['value']
                model = item['item']['name']
                fuel_type = item['item'].get('fuelType', 'N/A')
                engine_power = item['item'].get('vehicleEngine', {}).get('enginePower', {}).get('value', 'N/A')
                data.append({'model': model, 'price': price, 'currency': currency,'year': year, 'address': address,  'mileage': mileage, 'fuel_type': fuel_type, 'engine_power': engine_power})
# Print the data
for item in data:
    conn.execute('INSERT INTO autos (model, price, currency, year, address, mileage, fuel_type, engine_power) VALUES (?, ?, ?, ?, ?, ?, ?, ?)', 
                 (item['model'], item['price'], item['currency'], item['year'], item['address'], item['mileage'], item['fuel_type'], item['engine_power']))
    print(item)

# Close the driver
driver.quit()

# Änderungen speichern
conn.commit()

# Verbindung schließen
conn.close()

C:\Users\meyes\AppData\Local\Temp\ipykernel_12232\2345797812.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


{'model': 'AUDI A5 Cabriolet 2.0 TFSI S line Sport quattro S-tronic', 'price': '43900', 'currency': 'CHF', 'year': '09.2018', 'address': '8620 Wetzikon', 'mileage': "79'300", 'fuel_type': 'Benzin', 'engine_power': '252'}
{'model': 'AUDI A5 Sportback 2.0 TFSI quattro S-tronic', 'price': '19500', 'currency': 'CHF', 'year': '07.2012', 'address': '8156 Oberhasli', 'mileage': "126'950", 'fuel_type': 'Benzin', 'engine_power': '211'}
{'model': 'AUDI A5 Cabriolet 2.0 TFSI S-Line quattro S-tronic', 'price': '25900', 'currency': 'CHF', 'year': '12.2011', 'address': '9313 Muolen', 'mileage': "89'700", 'fuel_type': 'Benzin', 'engine_power': '211'}
{'model': 'AUDI A5 Coupé 3.0 TDI Sport quattro S-tronic', 'price': '39900', 'currency': 'CHF', 'year': '07.2017', 'address': '1950 SION', 'mileage': "29'000", 'fuel_type': 'Diesel', 'engine_power': '218'}
{'model': 'AUDI A5 Sportback 45 TFSI S-Line S-tronic quattro', 'price': '46500', 'currency': 'CHF', 'year': '06.2021', 'address': '8184 Bachenbülach', 

In [25]:
import sqlite3

# Verbindung zur Datenbank herstellen
conn = sqlite3.connect('autoscout24.db')

# Cursor erstellen
cur = conn.cursor()

# SELECT-Abfrage ausführen
cur.execute("SELECT * FROM autos")

# Ergebnisse abrufen
results = cur.fetchall()

# Ergebnisse ausgeben
for row in results:
    print(row)

# Verbindung schließen
conn.close()

(1, 'AUDI A5 Cabriolet 2.0 TFSI S line Sport quattro S-tronic', 43900, 'CHF', '09.2018', '8620 Wetzikon', "79'300", 'Benzin', 252)
(2, 'AUDI A5 Sportback 2.0 TFSI quattro S-tronic', 19500, 'CHF', '07.2012', '8156 Oberhasli', "126'950", 'Benzin', 211)
(3, 'AUDI A5 Cabriolet 2.0 TFSI S-Line quattro S-tronic', 25900, 'CHF', '12.2011', '9313 Muolen', "89'700", 'Benzin', 211)
(4, 'AUDI A5 Coupé 3.0 TDI Sport quattro S-tronic', 39900, 'CHF', '07.2017', '1950 SION', "29'000", 'Diesel', 218)
(5, 'AUDI A5 Sportback 45 TFSI S-Line S-tronic quattro', 46500, 'CHF', '06.2021', '8184 Bachenbülach', "37'075", 'Benzin', 265)
(6, 'AUDI A5 Cabriolet 45 TFSI S-Line S-tronic quattro', 77800, 'CHF', '06.2022', '2542 PIETERLEN', "1'300", 'Benzin', 265)
(7, 'AUDI A5 Cabriolet 2.0 TFSI quattro S-tronic', 9500, 'CHF', '08.2009', '3007 Bern', "174'000", 'Benzin', 211)
(8, 'AUDI A5 Sportback 3.0 TDI clean diesel quattro S-tronic', 29500, 'CHF', '02.2016', '8416 Flaach', "77'200", 'Diesel', 245)
(9, 'AUDI A5 Spor